In [82]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import pandas as pd


# 读取数据
df = pd.read_excel('D:/桌面/国赛C题代码/铅钡1.xlsx')

# 填补缺失值（可以用均值、中位数、0等方法填补）
df.fillna(0, inplace=True)

# 查看前几行数据
print(df.head())


   特征数据  二氧化硅(SiO2)  氧化钠(Na2O)  氧化钾(K2O)  氧化钙(CaO)  氧化镁(MgO)  氧化铝(Al2O3)  \
0     1       36.28        0.0      1.05      2.34      1.18        5.73   
1     1       20.14        0.0      0.00      1.48      0.00        1.34   
2     2        4.61        0.0      0.00      3.19      0.00        1.11   
3     1       33.59        0.0      0.21      3.51      0.71        2.69   
4     1       29.64        0.0      0.00      2.93      0.59        3.57   

   氧化铁(Fe2O3)  氧化铜(CuO)  氧化铅(PbO)  氧化钡(BaO)  五氧化二磷(P2O5)  氧化锶(SrO)  氧化锡(SnO2)  \
0        1.86      0.26     47.43      0.00         3.57      0.19        0.0   
1        0.00     10.41     28.68     31.23         3.59      0.37        0.0   
2        0.00      3.14     32.45     30.62         7.56      0.53        0.0   
3        0.00      4.93     25.39     14.61         9.38      0.37        0.0   
4        1.33      3.51     42.82      5.35         8.83      0.19        0.0   

   二氧化硫(SO2)  
0       0.00  
1       2.58  
2      15.0

In [83]:
# def map_wear_status(value):
#     if value == 1:
#         return '1'
#     elif value == 2:
#         return '严重风化'
#     elif value == 0:
#         return '无风化'
#     else:
#         return '未知'  # 对于无法匹配的情况，设为'未知'

# df['风化状态'] = df['特征数据'].apply(map_wear_status)



# 提取特征和标签
features = ['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', 
            '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', 
            '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)']

X = df[features]  # 特征数据
y = df.iloc[:, 0].values  # 标签数据

# 输出数据集的特征名称
print("数据集中定义的特征名称:", features)
print("数据集中定义的标签名称:", y)



数据集中定义的特征名称: ['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)']
数据集中定义的标签名称: [1 1 2 1 1 0 0 0 0 1 2 0 0 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1 0
 1 0 1 1 1 0 1 2 0 1 1 1]


In [84]:
# 将数据集拆分为训练集和测试集，测试集占总数据的30%，并设定随机种子以保证结果可重复
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 数据标准化
# 标准化有助于提高模型性能，使数据均值为0，标准差为1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 对训练数据进行标准化
X_test_scaled = scaler.transform(X_test)  # 对测试数据进行标准化（使用训练数据的均值和标准差）

# 定义超参数网格
# 设置不同的超参数组合来尝试，GridSearchCV 将会遍历这些组合
param_grid = {
    'max_depth': [3],  # 增加更大的深度
    'min_samples_split': [2, 5, 10, 20 ,None],   # 增加更多选项
    'min_samples_leaf': [1, 2, 4, 8, 10, 20, None],     # 增加更多选项
    'max_features': ['auto', 'sqrt', 'log2', None, 0.5, 0.75]  # 增加更多选项
}

# 创建并训练 GridSearchCV 实例
# GridSearchCV 将会遍历所有超参数组合，并使用交叉验证来评估每个组合的性能
grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(),  # 使用的模型
    param_grid=param_grid,               # 超参数网格
    cv=10,                                # 交叉验证的折数（分割数据进行验证）
    scoring='accuracy',                  # 评估标准，这里使用准确率
    n_jobs=-1,                           # 使用所有可用的CPU核心来加速计算
    verbose=1                            # 打印进度信息
)

# 使用训练数据进行网格搜索
grid_search.fit(X_train_scaled, y_train)

# 输出最佳参数和评分
# 打印网格搜索找到的最佳超参数组合和最佳模型的评分
print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

# 使用最佳参数训练最终模型
# 使用找到的最佳超参数配置来训练决策树分类器
best_dtc = grid_search.best_estimator_

# 评估最终模型
# 在测试集上评估最终模型的性能
accuracy = best_dtc.score(X_test_scaled, y_test)
print("Model Accuracy:", accuracy)

# 打印分类报告
# 分类报告包括精确率、召回率、F1分数等指标
print(classification_report(y, best_dtc.predict(X)))

Fitting 10 folds for each of 210 candidates, totalling 2100 fits


d:\Users\huang\anaconda3\envs\python310\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(


Best parameters found:  {'max_depth': 3, 'max_features': 0.75, 'min_samples_leaf': 2, 'min_samples_split': 10}
Best score:  0.85
Model Accuracy: 0.7333333333333333
              precision    recall  f1-score   support

           0       0.47      1.00      0.64        23
           1       0.00      0.00      0.00        23
           2       0.00      0.00      0.00         3

    accuracy                           0.47        49
   macro avg       0.16      0.33      0.21        49
weighted avg       0.22      0.47      0.30        49



d:\Users\huang\anaconda3\envs\python310\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
900 fits failed out of a total of 2100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
176 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Users\huang\anaconda3\envs\python310\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Users\huang\anaconda3\envs\python310\lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "d:\Users\huang\anaconda3\envs\python310\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_

In [85]:
from sklearn.tree import export_graphviz

# 确保使用正确的特征名称
feature_names = ['二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)', 
                  '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)', 
                  '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)']

# 导出决策树模型到 .dot 文件
export_graphviz(best_dtc, out_file='D:/桌面/国赛C题代码/classify_tree.dot', 
                feature_names=feature_names, 
                class_names=best_dtc.classes_.astype(str), 
                filled=True, 
                rounded=True, 
                special_characters=True)

#在命令行输入 运行
# dot -Tpng classify_tree.dot -o classify_tree.png
# -Tpng 指定输出格式为 PNG。
# -o 指定输出文件的路径和名称。
